In [61]:
gbm_params = {
    'n_estimators' : 1500, # 500,  
    'max_depth' : -1,
    'learning_rate': 0.05,
    'bagging_fraction': 0.7, 
    
    'feature_fraction' : 0.9,
    'bagging_freq': 5,
    #'subsample' : 0.1,  # 
    #'subsample_freq' : 1,
    'num_leaves' : 31,
    'metric':'rmse',
    #'lambda_l1' : 1,  # Try defaults
    #'lambda_l2': 1, # Try defaults
    'verbose': 100
}

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from tqdm import tqdm
from datetime import date 
import holidays
import lightgbm as lgb


warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [63]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [64]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.float32, 'year_built': np.float16, 'floor_count': np.float16},
    'weather_test' : {'site_id': np.int8, 'air_temperature': np.float16, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16},
    'weather_train' : {'site_id': np.int8, 'air_temperature': np.float16, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16}
}

def loadFile(name):
    for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
        if path.exists(dir_path + name + '.csv'):
            return  ConvertToDatetime().transform(
                pd.read_csv(dir_path + name + '.csv', dtype=file_dtype[name]))
        


In [65]:
building = loadFile('building_metadata')
weather_train = loadFile('weather_train')
weather_test = loadFile('weather_test')
train = loadFile('train')
test = loadFile('test')



In [66]:
# align weather
def weatherSiteOffsets():
    weather = pd.concat([weather_train,weather_test],ignore_index=True)
    weather['timestamp'] = pd.to_datetime(weather['timestamp'])
    weather_key = ['site_id', 'timestamp']

    temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

    # calculate ranks of hourly temperatures within date/site_id chunks
    temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')
    
    # create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
    df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

    # Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
    site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
    site_ids_offsets.index.name = 'site_id'
    return site_ids_offsets

site_time_offsets_df = weatherSiteOffsets()

def alignWeather(df):
    df['offset'] = df.site_id.map(site_time_offsets_df)
    df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
    df['timestamp'] = df['timestamp_aligned']
    del df['timestamp_aligned'], df['offset']
    gc.collect()
    return df

weather_train = alignWeather(weather_train)
weather_test = alignWeather(weather_test)

del site_time_offsets_df
gc.collect()

7

In [67]:
if path.exists('../input/ashrae-energy-prediction/weather_train_s_radiation.pickle'):
    weather_train['s_radiation'] =  pd.read_pickle('../input/ashrae-energy-prediction/weather_train_s_radiation.pickle')
    weather_test['s_radiation'] =  pd.read_pickle('../input/ashrae-energy-prediction/weather_test_s_radiation.pickle')

print(weather_train.head())
    

   site_id           timestamp  air_temperature  cloud_coverage  \
0        0 2015-12-31 19:00:00        25.000000             6.0   
1        0 2015-12-31 20:00:00        24.406250             NaN   
2        0 2015-12-31 21:00:00        22.796875             2.0   
3        0 2015-12-31 22:00:00        21.093750             2.0   
4        0 2015-12-31 23:00:00        20.000000             2.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0         20.00000                NaN              1019.5             0.0   
1         21.09375               -1.0              1020.0            70.0   
2         21.09375                0.0              1020.0             0.0   
3         20.59375                0.0              1020.0             0.0   
4         20.00000               -1.0              1020.0           250.0   

   wind_speed  s_radiation  
0    0.000000          0.0  
1    1.500000          0.0  
2    0.000000          0.0  
3    0.000000     

In [68]:
# TODO: do something with race precipitation is coded as a “-1” value
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113103#latest-664978
# Test out below
def cleanPrecipDepth(df):
    df.loc[df['precip_depth_1_hr'] == -1, 'precip_depth_1_hr'] = 0.25
    return df

#weather_train = cleanPrecipDepth(weather_train)
#weather_test = cleanPrecipDepth(weather_test)

#print(weather_train['precip_depth_1_hr'].value_counts().sort_index())

In [69]:
# See holiday notebook to generate
holiday_df = None
if path.exists('../input/holiday-pickle/holiday_df.pickle'):
    holiday_df = pd.read_pickle('../input/holiday-pickle/holiday_df.pickle')
elif path.exists('../input/ashrae-energy-prediction/holiday_df.pickle'):
    holiday_df = pd.read_pickle('../input/ashrae-energy-prediction/holiday_df.pickle')

In [70]:
class MeterReadingLog1p(TransformerMixin):
  
    def transform(self, df, **transform_params):
        if 'meter_reading' in df.columns:
            df['meter_reading_log1p'] = np.log1p(df['meter_reading'])
            df = df.drop('meter_reading', axis=1)
        return df
    
    def fit(self, X, y=None, **fit_params):
        return self
print(train.sample(20, random_state=42))
print(MeterReadingLog1p().transform(train.sample(20, random_state=42)))
gc.collect()

          building_id  meter           timestamp  meter_reading
14245562         1324      1 2016-09-16 16:00:00       0.000000
1282718          1013      0 2016-01-24 06:00:00      32.000099
13883790          229      1 2016-09-10 07:00:00     567.655029
4781820           217      3 2016-04-01 01:00:00       0.000000
10415393         1434      0 2016-07-10 04:00:00      65.750000
1057008          1047      0 2016-01-20 04:00:00      90.983299
4507399           911      1 2016-03-26 20:00:00     295.063995
19478829         1039      0 2016-12-18 23:00:00      16.900000
8955615           265      0 2016-06-14 06:00:00     128.369995
13799839          896      0 2016-09-08 19:00:00     300.000000
15647011          973      0 2016-10-11 11:00:00     247.000000
2524294           813      0 2016-02-16 08:00:00      10.958300
10016102          870      0 2016-07-03 02:00:00       4.166700
3915750           898      0 2016-03-15 03:00:00      40.000000
17217526          903      0 2016-11-08 

142

In [71]:
class BuildingMeterDescDF(TransformerMixin):

    def transform(self, df, **transform_params):
        building_df = pd.DataFrame(data={'building_id':train['building_id'].unique()})
        for i in range(4):
            group = df[train['meter'] == i].groupby(['building_id'])['meter_reading_log1p']
            #print(group.head())
            building_mean = group.mean().astype(np.float16)
            building_median = group.median().astype(np.float16)
            building_min = group.min().astype(np.float16)
            building_max = group.max().astype(np.float16)
            building_std = group.std().astype(np.float16)
            building_df['meter_' + str(i) +'_mean'] = building_df['building_id'].map(building_mean).fillna(0)
            building_df['meter_' + str(i) +'_median'] = building_df['building_id'].map(building_median).fillna(0)
            building_df['meter_' + str(i) +'_min'] = building_df['building_id'].map(building_min).fillna(0)
            building_df['meter_' + str(i) +'_max'] = building_df['building_id'].map(building_max).fillna(0)
            building_df['meter_' + str(i) +'_std'] = building_df['building_id'].map(building_std).fillna(0)
            del group
            gc.collect()
        return building_df 
    def fit(self, X, y=None, **fit_params):
        return self

if 'meter_0_mean' not in test.columns:
    building_meter_desc_DF = BuildingMeterDescDF().transform(MeterReadingLog1p().transform(train))
    train = train.merge(building_meter_desc_DF, on='building_id', how='left')
    test = test.merge(building_meter_desc_DF, on='building_id', how='left')
    del building_meter_desc_DF
    
print(test.sample(20, random_state=42))
gc.collect()

            row_id  building_id  meter           timestamp  meter_0_mean  \
3573457    3573457          173      0 2017-02-02 08:00:00      4.832031   
8315486    8315486          222      1 2018-12-18 00:00:00      2.617188   
40305643  40305643         1354      2 2018-06-14 12:00:00      0.000000   
16083617  16083617          712      0 2018-08-23 23:00:00      1.302734   
37204119  37204119         1344      2 2017-01-02 00:00:00      5.789062   
32144852  32144852         1119      1 2018-12-30 19:00:00      4.425781   
5105044    5105044          249      0 2017-09-11 03:00:00      5.921875   
36982844  36982844         1303      1 2018-11-30 19:00:00      5.328125   
20487823  20487823          945      2 2017-04-23 08:00:00      5.406250   
8404196    8404196          217      1 2018-12-30 19:00:00      5.562500   
6889602    6889602          241      0 2018-05-26 10:00:00      3.972656   
16963616  16963616          784      0 2017-11-21 15:00:00      6.503906   
39666699  39

73

In [72]:
# "As you can see above, this data looks weired until May 20. It is 
# reported in this discussion by @barnwellguy that All electricity
# meter is 0 until May 20 for site_id == 0. Let's remove these data 
# from training data."
# https://www.kaggle.com/kaushal2896/ashrae-eda-fe-lightgbm-1-13
class RmS0M0(TransformerMixin):
  
    def transform(self, df, **transform_params):
        return df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

    def fit(self, X, y=None, **fit_params):
        return self
    


In [73]:
# TODO: write filter to remove any 0 meter reading that continue more than N days (try 3)
# Also we need to account for this by meter

In [74]:
# TODO: try both for direction
class ImputeWeather(TransformerMixin):

    def __init__(self, method:str='linear', gap_limit:int=None, limit_direction:str='forward'):
        self._method = method
        self._gap_limit = gap_limit
        self._limit_direction = limit_direction
        
    def transform(self, weather_df, **transform_params):
        grouped_weather_df = weather_df.groupby('site_id').apply(lambda group: group.interpolate(method=self._method, limit=self._gap_limit, limit_direction=self._limit_direction))
        if 'cloud_coverage' in grouped_weather_df.columns:
            grouped_weather_df['cloud_coverage'] = grouped_weather_df['cloud_coverage'].round(decimals=0).clip(0,8)
        grouped_weather_df.reset_index(inplace=True)
        weather_df = grouped_weather_df.drop(['index'], axis=1)
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
print(weather_train.head(20))
print(ImputeWeather().transform(weather_train.head(20)))

    site_id           timestamp  air_temperature  cloud_coverage  \
0         0 2015-12-31 19:00:00        25.000000             6.0   
1         0 2015-12-31 20:00:00        24.406250             NaN   
2         0 2015-12-31 21:00:00        22.796875             2.0   
3         0 2015-12-31 22:00:00        21.093750             2.0   
4         0 2015-12-31 23:00:00        20.000000             2.0   
5         0 2016-01-01 00:00:00        19.406250             NaN   
6         0 2016-01-01 01:00:00        21.093750             6.0   
7         0 2016-01-01 02:00:00        21.093750             NaN   
8         0 2016-01-01 03:00:00        20.593750             NaN   
9         0 2016-01-01 04:00:00        21.093750             NaN   
10        0 2016-01-01 05:00:00        21.093750             NaN   
11        0 2016-01-01 06:00:00        20.593750             NaN   
12        0 2016-01-01 07:00:00        18.906250             6.0   
13        0 2016-01-01 08:00:00        20.000000

In [75]:
# TODO: rename to rolling
class AddWeatherLags(TransformerMixin):
    
    def __init__(self, window):
        self._window = window
        
    def transform(self, weather_df, **transform_params):
        group_df = weather_df.groupby(['site_id'])
        cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
        if 's_radiation' in weather_df.columns:
            cols.append('s_radiation')
        rolled = group_df[cols].rolling(window=self._window, min_periods=0)
        lag_mean = rolled.mean().reset_index().astype(np.float16)
        lag_max = rolled.max().reset_index().astype(np.float16)
        lag_min = rolled.min().reset_index().astype(np.float16)
        lag_std = rolled.std().reset_index().astype(np.float16)
        for col in cols:
            weather_df[f'{col}_mean_lag{self._window}'] = lag_mean[col]
            weather_df[f'{col}_max_lag{self._window}'] = lag_max[col]
            weather_df[f'{col}_min_lag{self._window}'] = lag_min[col]
            weather_df[f'{col}_std_lag{self._window}'] = lag_std[col]
        del group_df, rolled
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
    
print(AddWeatherLags(72).transform(weather_train.head(20)))

    site_id           timestamp  air_temperature  cloud_coverage  \
0         0 2015-12-31 19:00:00        25.000000             6.0   
1         0 2015-12-31 20:00:00        24.406250             NaN   
2         0 2015-12-31 21:00:00        22.796875             2.0   
3         0 2015-12-31 22:00:00        21.093750             2.0   
4         0 2015-12-31 23:00:00        20.000000             2.0   
5         0 2016-01-01 00:00:00        19.406250             NaN   
6         0 2016-01-01 01:00:00        21.093750             6.0   
7         0 2016-01-01 02:00:00        21.093750             NaN   
8         0 2016-01-01 03:00:00        20.593750             NaN   
9         0 2016-01-01 04:00:00        21.093750             NaN   
10        0 2016-01-01 05:00:00        21.093750             NaN   
11        0 2016-01-01 06:00:00        20.593750             NaN   
12        0 2016-01-01 07:00:00        18.906250             6.0   
13        0 2016-01-01 08:00:00        20.000000

[20 rows x 42 columns]


In [76]:
class AddBuilding(TransformerMixin):

    def __init__(self, building_df):
        self._b_df = building_df
        
    def transform(self, df, **transform_params):
        return df.merge(_b_df, on='building_id', how='left')

    def fit(self, X, y=None, **fit_params):
        return self

In [77]:
# TODO: try rolling with power

In [78]:
    
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114483#latest-660771
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114874#latest-660970
class AddHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        if holiday_df is not None:
            df = df.merge(holiday_df, on=['building_id','meter','timestamp','site_id'], how='left')
        else:
            print("Warning: Holiday DF is missing")
        return df

    def fit(self, X, y=None, **fit_params):
        return self
# Test 
print(AddHolidays().transform(train.head(2000).merge(building, on='building_id', how='left'))[['building_id','timestamp','holiday']])

      building_id  timestamp         holiday
0               0 2016-01-01  New Year's Day
1               1 2016-01-01  New Year's Day
2               2 2016-01-01  New Year's Day
3               3 2016-01-01  New Year's Day
4               4 2016-01-01  New Year's Day
5               5 2016-01-01  New Year's Day
6               6 2016-01-01  New Year's Day
7               7 2016-01-01  New Year's Day
8               8 2016-01-01  New Year's Day
9               9 2016-01-01  New Year's Day
10             10 2016-01-01  New Year's Day
11             11 2016-01-01  New Year's Day
12             12 2016-01-01  New Year's Day
13             13 2016-01-01  New Year's Day
14             14 2016-01-01  New Year's Day
15             15 2016-01-01  New Year's Day
16             16 2016-01-01  New Year's Day
17             17 2016-01-01  New Year's Day
18             18 2016-01-01  New Year's Day
19             19 2016-01-01  New Year's Day
20             20 2016-01-01  New Year's Day
21        

In [79]:
class RmHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        if holiday_df is not None:
            df = df.merge(holiday_df, on=['building_id','meter','timestamp','site_id'], how='left')
            df = df.drop(df[df['holiday'].notnull()].index)
            df = df.drop(['holiday'], axis=1)
            gc.collect()
        else:
            print("Warning: Holiday DF is missing")
        return df

    def fit(self, X, y=None, **fit_params):
        return self

# Test you should see the new years removed
print(RmHolidays().transform(train.head(100000).merge(building, on='building_id', how='left')))

       building_id  meter           timestamp  meter_reading  \
55121            0      0 2016-01-02 00:00:00       0.000000   
55122            1      0 2016-01-02 00:00:00       0.000000   
55123            2      0 2016-01-02 00:00:00       0.000000   
55124            3      0 2016-01-02 00:00:00       0.000000   
55125            4      0 2016-01-02 00:00:00       0.000000   
55126            5      0 2016-01-02 00:00:00       0.000000   
55127            6      0 2016-01-02 00:00:00       0.000000   
55128            7      0 2016-01-02 00:00:00       0.000000   
55129            8      0 2016-01-02 00:00:00       0.000000   
55130            9      0 2016-01-02 00:00:00       0.000000   
55131           10      0 2016-01-02 00:00:00       0.000000   
55132           11      0 2016-01-02 00:00:00       0.000000   
55133           12      0 2016-01-02 00:00:00       0.000000   
55134           13      0 2016-01-02 00:00:00       0.000000   
55135           14      0 2016-01-02 00:

[41839 rows x 30 columns]


In [80]:
class SetCatTypes(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['primary_use']= df['primary_use'].astype('category')
        df['meter'] = df["meter"].astype('category')
        df['site_id'] = df["site_id"].astype('category')
        df['building_id'] = df['building_id'].astype('category')
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [81]:
class ImputeCloudCoverage(TransformerMixin):
        
    def transform(self, df, **transform_params):
        # set age of building to mediam of site_id
        # else if set ot overall median
        median = df['cloud_coverage'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['cloud_coverage'].median().items():
            # TODO add in +9 as a NAN
            if not np.isnan(i_median):
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = i_median
            else:
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = median
        df['cloud_coverage'] = np.uint8(df['cloud_coverage'])
        df['cloud_coverage'] = df['cloud_coverage']
        del median
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [82]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(building.head(20)['square_feet'])

0       7432.0
1       2720.0
2       5376.0
3      23685.0
4     116607.0
5       8000.0
6      27926.0
7     121074.0
8      60809.0
9      27000.0
10    370773.0
11     49073.0
12     37100.0
13     99380.0
14     86250.0
15     83957.0
16     54644.0
17     15250.0
18    111891.0
19     18717.0
Name: square_feet, dtype: float32


In [83]:
# TODO: Play with scaling cloud coverage

In [84]:
class CloudTimeCat(TransformerMixin):
        
    def transform(self, df, **transform_params):
        tempDf = df[['cloud_coverage', 'hour']].astype('int')
        tempDf['cloud_coverage'] = (tempDf['cloud_coverage']).astype('int')
        tempDf['hour'] = (tempDf['hour']).astype('int')
        tempDf = tempDf.astype('str')
        df['cloud_time_cat'] = 'c' + tempDf['cloud_coverage'] + 't' + tempDf['hour']
        df['cloud_time_cat'] = df['cloud_time_cat'].astype('category')
        del tempDf
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [85]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [86]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        # revisit the choice of median vs anything else
        year_built_median = df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        del year_built_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [87]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [88]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        # TODO: try week of year as numerical 
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('category') # vs weekend?
        #df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['dayofweek_hour'] = (df['timestamp'].dt.dayofweek * 24) + df['timestamp'].dt.hour
        #df['dayofweek_hour'] = df['dayofweek_hour'].astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('uint8')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [89]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [90]:
class FillMean(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].mean())
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [91]:
class FillZeros(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(0)
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [92]:
class FillMedian(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].median())
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [93]:
class FillPopular(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].value_counts()[0])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [94]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [95]:
class GC(TransformerMixin):
        
    def transform(self, df, **transform_params):
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [96]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred.clip(0)))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)

def lbm_rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

# rob's custome function to do RMSLE while in the log1p space
def lbm_rmslee(y_true, y_pred):
    return 'RMSLEE', np.sqrt(np.mean(np.power(y_pred - y_true, 2))), False



In [97]:
weather_pipes = Pipeline(
    steps=[
        #('convertToDatetime', ConvertToDatetime()),
        ('imputeWeather', ImputeWeather()),
        ('fillMean',FillMean(['air_temperature','dew_temperature'
                              , 'precip_depth_1_hr', 'sea_level_pressure'])),
        ('imputeCloudCoverage', ImputeCloudCoverage()),
        ('addWeatherLags3', AddWeatherLags(3)),
        ('addWeatherLags72', AddWeatherLags(72)),
    ]
)

building_pipes = Pipeline(
    steps=[
        ('logSquareFeet', LogSquareFeet()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('fillMean',FillMean(['floor_count'])),
        ('dropClos', DropCols(['square_feet', 'year_built'])),
    ]
)


# pre_a_pipes is for preprocessing that doesn't change impute
# values
x_pipes = Pipeline(
    steps=[
        #('markNans',MarkNaNs()),
        #('convertToDatetime', ConvertToDatetime()),
        ('meterReadingLog1p',MeterReadingLog1p()),
        ('rmS0M0', RmS0M0()),
        #('addHolidays', AddHolidays()),
        #('rmHolidays', RmHolidays()), called manually in fold
        ('addRelativeHumidity',AddRelativeHumidity()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('setCatTypes', SetCatTypes()),
        ('fillMean',FillMean([])),
        ('fillZeros',FillZeros([])),
        ('dropCols', DropCols(['timestamp'])),
        ('GC', GC())
    ]
)

In [98]:
def getInFoldXY(train_index):
    X = train.iloc[train_index]
    X_buildings = building[building['building_id'].isin(X['building_id'].unique())]
    X = X.merge(building_pipes.transform(X_buildings), on='building_id', how='left')
    X_weather = weather_train[
        (weather_train['site_id'].isin(X['site_id'].unique())) 
         & (weather_train['timestamp'].isin(X['timestamp'].unique())) 
    ]
    X = x_pipes.transform(
        RmHolidays().transform(
            X.merge(weather_pipes.transform(X_weather), how='left')))
    return X

sample_train_X = getInFoldXY(train.sample(n=20,  random_state=42).index)
print(sample_train_X)

   building_id meter  meter_reading_log1p  meter_0_mean  meter_0_median  \
0         1324     1             0.000000      3.050781        2.708984   
1         1013     0             3.496511      4.003906        3.798828   
2          229     1             6.343274      5.656250        5.742188   
3          217     3             0.000000      5.562500        5.515625   
4         1434     0             4.200954      4.355469        4.332031   
5         1047     0             4.521607      4.914062        5.015625   
6          911     1             5.690576      5.546875        5.699219   
7         1039     0             2.884801      3.289062        3.130859   
8          265     0             4.862677      4.785156        4.734375   
9          896     0             5.707110      5.402344        5.562500   
10         973     0             5.513429      4.976562        5.035156   
11         813     0             2.481426      2.427734        2.492188   
12         870     0     

In [99]:
# this stratified strategy from
# https://www.kaggle.com/isaienkov/lightgbm-fe-1-19/notebook
# trying no shuffle https://www.kaggle.com/c/ashrae-energy-prediction/discussion/115851#latest-666115
folds = 5
kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=42)


In [100]:
## cross val multipule meter models
def cvTrainMeterEnsemble(train, gbm_params):
    meter_models = [ [], [], [], [] ]
    for train_index, val_index in kf.split(train, train['building_id']):
        f_train = getInFoldXY(train_index)
        f_val = getInFoldXY(val_index)
        for i in range(4):
            f_train_m = f_train[f_train['meter'] == i]
            f_val_m = f_val[f_val['meter'] == i]
            gbm_params_m = gbm_params
            if i == 0:
                gbm_params_m['n_estimators']=1500
            elif i == 1:
                gbm_params_m['learning_rate']=0.05
                gbm_params_m['bagging_fraction']=0.05
            elif i == 2:
                gbm_params_m['learning_rate']=0.05
                gbm_params_m['bagging_fraction']=0.08
            else:
                gbm_params_m['learning_rate']=0.03
                gbm_params_m['bagging_fraction']=0.09                
            gbm = LGBMRegressor(**gbm_params_m)
            gbm.fit(f_train_m.drop('meter_reading_log1p', axis=1), f_train_m['meter_reading_log1p'],
                eval_set=[(f_val_m.drop('meter_reading_log1p', axis=1), f_val_m['meter_reading_log1p'])],
                # https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114722#latest-660848
                # eval_metric=lbm_rmslee,
                early_stopping_rounds=20)
            meter_models[i].append(gbm)
            del f_train_m, f_val_m, gbm
            gc.collect()
        del f_train, f_val
        gc.collect()
    return meter_models


In [101]:
## cross single mdoels

def cvTrainSingleEnsemble(train, gbm_params):
    meter_models = [ [] ] # same to use same print out code later
    for train_index, val_index in kf.split(train, train['building_id']):
        f_train = getInFoldXY(train_index)
        f_val = getInFoldXY(val_index)
        gbm.fit(f_train.drop('meter_reading_log1p', axis=1), f_train['meter_reading_log1p'],
            eval_set=[(f_val.drop('meter_reading_log1p', axis=1), f_val['meter_reading_log1p'])],
            # https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114722#latest-660848
            # eval_metric=lbm_rmslee,
            early_stopping_rounds=20)
        meter_models[i].append(gbm)
        del f_train, f_val
        gc.collect()
    return meter_models


In [102]:
%%time
meter_models = cvTrainMeterEnsemble(train, gbm_params)

[1]	valid_0's rmse: 1.50434
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.44478
[3]	valid_0's rmse: 1.38731
[4]	valid_0's rmse: 1.33401
[5]	valid_0's rmse: 1.28336
[6]	valid_0's rmse: 1.2369
[7]	valid_0's rmse: 1.19276
[8]	valid_0's rmse: 1.15236
[9]	valid_0's rmse: 1.11553
[10]	valid_0's rmse: 1.08051
[11]	valid_0's rmse: 1.04648
[12]	valid_0's rmse: 1.01512
[13]	valid_0's rmse: 0.985768
[14]	valid_0's rmse: 0.959911
[15]	valid_0's rmse: 0.934704
[16]	valid_0's rmse: 0.91155
[17]	valid_0's rmse: 0.891096
[18]	valid_0's rmse: 0.871692
[19]	valid_0's rmse: 0.853241
[20]	valid_0's rmse: 0.836161
[21]	valid_0's rmse: 0.820708
[22]	valid_0's rmse: 0.806781
[23]	valid_0's rmse: 0.79404
[24]	valid_0's rmse: 0.782006
[25]	valid_0's rmse: 0.771671
[26]	valid_0's rmse: 0.761541
[27]	valid_0's rmse: 0.755585
[28]	valid_0's rmse: 0.746369
[29]	valid_0's rmse: 0.741007
[30]	valid_0's rmse: 0.735038
[31]	valid_0's rmse: 0.729249
[32]	valid_0's rmse: 0.725659
[33

[185]	valid_0's rmse: 1.23755
[186]	valid_0's rmse: 1.23716
[187]	valid_0's rmse: 1.23704
[188]	valid_0's rmse: 1.23681
[189]	valid_0's rmse: 1.23667
[190]	valid_0's rmse: 1.2361
[191]	valid_0's rmse: 1.2353
[192]	valid_0's rmse: 1.23467
[193]	valid_0's rmse: 1.2346
[194]	valid_0's rmse: 1.23444
[195]	valid_0's rmse: 1.2343
[196]	valid_0's rmse: 1.23448
[197]	valid_0's rmse: 1.23449
[198]	valid_0's rmse: 1.23448
[199]	valid_0's rmse: 1.23448
[200]	valid_0's rmse: 1.23435
[201]	valid_0's rmse: 1.23439
[202]	valid_0's rmse: 1.23464
[203]	valid_0's rmse: 1.23486
[204]	valid_0's rmse: 1.23482
[205]	valid_0's rmse: 1.23486
[206]	valid_0's rmse: 1.23501
[207]	valid_0's rmse: 1.2351
[208]	valid_0's rmse: 1.23564
[209]	valid_0's rmse: 1.23585
[210]	valid_0's rmse: 1.23581
[211]	valid_0's rmse: 1.23485
[212]	valid_0's rmse: 1.23383
[213]	valid_0's rmse: 1.23325
[214]	valid_0's rmse: 1.2328
[215]	valid_0's rmse: 1.23266
[216]	valid_0's rmse: 1.23243
[217]	valid_0's rmse: 1.23215
[218]	valid_0's 

[90]	valid_0's rmse: 1.48751
[91]	valid_0's rmse: 1.48766
[92]	valid_0's rmse: 1.48785
[93]	valid_0's rmse: 1.48785
[94]	valid_0's rmse: 1.48814
[95]	valid_0's rmse: 1.48829
[96]	valid_0's rmse: 1.48822
[97]	valid_0's rmse: 1.48748
[98]	valid_0's rmse: 1.48825
[99]	valid_0's rmse: 1.48811
[100]	valid_0's rmse: 1.48803
[101]	valid_0's rmse: 1.48798
[102]	valid_0's rmse: 1.48818
[103]	valid_0's rmse: 1.48757
[104]	valid_0's rmse: 1.48723
[105]	valid_0's rmse: 1.48714
[106]	valid_0's rmse: 1.48632
[107]	valid_0's rmse: 1.48502
[108]	valid_0's rmse: 1.4846
[109]	valid_0's rmse: 1.48408
[110]	valid_0's rmse: 1.48396
[111]	valid_0's rmse: 1.48401
[112]	valid_0's rmse: 1.48402
[113]	valid_0's rmse: 1.48379
[114]	valid_0's rmse: 1.48351
[115]	valid_0's rmse: 1.48303
[116]	valid_0's rmse: 1.48281
[117]	valid_0's rmse: 1.48258
[118]	valid_0's rmse: 1.48295
[119]	valid_0's rmse: 1.48307
[120]	valid_0's rmse: 1.483
[121]	valid_0's rmse: 1.48298
[122]	valid_0's rmse: 1.48234
[123]	valid_0's rmse: 1

[103]	valid_0's rmse: 1.54444
[104]	valid_0's rmse: 1.54368
[105]	valid_0's rmse: 1.54305
[106]	valid_0's rmse: 1.54172
[107]	valid_0's rmse: 1.54058
[108]	valid_0's rmse: 1.53962
[109]	valid_0's rmse: 1.53907
[110]	valid_0's rmse: 1.53718
[111]	valid_0's rmse: 1.53652
[112]	valid_0's rmse: 1.5355
[113]	valid_0's rmse: 1.53538
[114]	valid_0's rmse: 1.53522
[115]	valid_0's rmse: 1.53438
[116]	valid_0's rmse: 1.53269
[117]	valid_0's rmse: 1.53175
[118]	valid_0's rmse: 1.53091
[119]	valid_0's rmse: 1.52994
[120]	valid_0's rmse: 1.52789
[121]	valid_0's rmse: 1.52643
[122]	valid_0's rmse: 1.52466
[123]	valid_0's rmse: 1.52341
[124]	valid_0's rmse: 1.52245
[125]	valid_0's rmse: 1.52227
[126]	valid_0's rmse: 1.52148
[127]	valid_0's rmse: 1.52145
[128]	valid_0's rmse: 1.52103
[129]	valid_0's rmse: 1.52163
[130]	valid_0's rmse: 1.52138
[131]	valid_0's rmse: 1.52071
[132]	valid_0's rmse: 1.51959
[133]	valid_0's rmse: 1.51894
[134]	valid_0's rmse: 1.51824
[135]	valid_0's rmse: 1.51757
[136]	valid

[80]	valid_0's rmse: 0.560263
[81]	valid_0's rmse: 0.558993
[82]	valid_0's rmse: 0.557039
[83]	valid_0's rmse: 0.555328
[84]	valid_0's rmse: 0.554205
[85]	valid_0's rmse: 0.552767
[86]	valid_0's rmse: 0.551876
[87]	valid_0's rmse: 0.550939
[88]	valid_0's rmse: 0.549572
[89]	valid_0's rmse: 0.548695
[90]	valid_0's rmse: 0.547728
[91]	valid_0's rmse: 0.546623
[92]	valid_0's rmse: 0.545769
[93]	valid_0's rmse: 0.544689
[94]	valid_0's rmse: 0.544011
[95]	valid_0's rmse: 0.54325
[96]	valid_0's rmse: 0.542732
[97]	valid_0's rmse: 0.54218
[98]	valid_0's rmse: 0.541149
[99]	valid_0's rmse: 0.540204
[100]	valid_0's rmse: 0.539609
[101]	valid_0's rmse: 0.538949
[102]	valid_0's rmse: 0.538584
[103]	valid_0's rmse: 0.537836
[104]	valid_0's rmse: 0.537193
[105]	valid_0's rmse: 0.53677
[106]	valid_0's rmse: 0.536008
[107]	valid_0's rmse: 0.535639
[108]	valid_0's rmse: 0.535244
[109]	valid_0's rmse: 0.534895
[110]	valid_0's rmse: 0.534435
[111]	valid_0's rmse: 0.534213
[112]	valid_0's rmse: 0.533734


...

[168]	valid_0's rmse: 1.48061
[169]	valid_0's rmse: 1.48066
[170]	valid_0's rmse: 1.48032
[171]	valid_0's rmse: 1.48011
[172]	valid_0's rmse: 1.47979
[173]	valid_0's rmse: 1.47954
[174]	valid_0's rmse: 1.47941
[175]	valid_0's rmse: 1.47925
[176]	valid_0's rmse: 1.47921
[177]	valid_0's rmse: 1.47878
[178]	valid_0's rmse: 1.47889
[179]	valid_0's rmse: 1.47913
[180]	valid_0's rmse: 1.47925
[181]	valid_0's rmse: 1.47945
[182]	valid_0's rmse: 1.47957
[183]	valid_0's rmse: 1.47956
[184]	valid_0's rmse: 1.47962
[185]	valid_0's rmse: 1.47969
[186]	valid_0's rmse: 1.47903
[187]	valid_0's rmse: 1.47903
[188]	valid_0's rmse: 1.47887
[189]	valid_0's rmse: 1.47837
[190]	valid_0's rmse: 1.47803
[191]	valid_0's rmse: 1.47794
[192]	valid_0's rmse: 1.47798
[193]	valid_0's rmse: 1.47789
[194]	valid_0's rmse: 1.47788
[195]	valid_0's rmse: 1.47797
[196]	valid_0's rmse: 1.47776
[197]	valid_0's rmse: 1.47738
[198]	valid_0's rmse: 1.4773
[199]	valid_0's rmse: 1.47703
[200]	valid_0's rmse: 1.47706
[201]	valid

[196]	valid_0's rmse: 1.57464
[197]	valid_0's rmse: 1.57467
[198]	valid_0's rmse: 1.5746
[199]	valid_0's rmse: 1.57445
[200]	valid_0's rmse: 1.57447
[201]	valid_0's rmse: 1.57451
[202]	valid_0's rmse: 1.57412
[203]	valid_0's rmse: 1.5742
[204]	valid_0's rmse: 1.57389
[205]	valid_0's rmse: 1.57394
[206]	valid_0's rmse: 1.5742
[207]	valid_0's rmse: 1.57406
[208]	valid_0's rmse: 1.57389
[209]	valid_0's rmse: 1.57403
[210]	valid_0's rmse: 1.57419
[211]	valid_0's rmse: 1.5744
[212]	valid_0's rmse: 1.57406
[213]	valid_0's rmse: 1.5738
[214]	valid_0's rmse: 1.5734
[215]	valid_0's rmse: 1.57351
[216]	valid_0's rmse: 1.57224
[217]	valid_0's rmse: 1.57233
[218]	valid_0's rmse: 1.57269
[219]	valid_0's rmse: 1.57245
[220]	valid_0's rmse: 1.5726
[221]	valid_0's rmse: 1.57227
[222]	valid_0's rmse: 1.57133
[223]	valid_0's rmse: 1.57128
[224]	valid_0's rmse: 1.57116
[225]	valid_0's rmse: 1.57111
[226]	valid_0's rmse: 1.57105
[227]	valid_0's rmse: 1.57098
[228]	valid_0's rmse: 1.57093
[229]	valid_0's r

[114]	valid_0's rmse: 1.60411
[115]	valid_0's rmse: 1.60358
[116]	valid_0's rmse: 1.6026
[117]	valid_0's rmse: 1.60203
[118]	valid_0's rmse: 1.60175
[119]	valid_0's rmse: 1.60119
[120]	valid_0's rmse: 1.60056
[121]	valid_0's rmse: 1.60032
[122]	valid_0's rmse: 1.60016
[123]	valid_0's rmse: 1.59954
[124]	valid_0's rmse: 1.59922
[125]	valid_0's rmse: 1.59914
[126]	valid_0's rmse: 1.59801
[127]	valid_0's rmse: 1.59732
[128]	valid_0's rmse: 1.59677
[129]	valid_0's rmse: 1.59637
[130]	valid_0's rmse: 1.59574
[131]	valid_0's rmse: 1.59534
[132]	valid_0's rmse: 1.59513
[133]	valid_0's rmse: 1.59477
[134]	valid_0's rmse: 1.59445
[135]	valid_0's rmse: 1.5942
[136]	valid_0's rmse: 1.59419
[137]	valid_0's rmse: 1.59429
[138]	valid_0's rmse: 1.59442
[139]	valid_0's rmse: 1.59411
[140]	valid_0's rmse: 1.59383
[141]	valid_0's rmse: 1.59366
[142]	valid_0's rmse: 1.59351
[143]	valid_0's rmse: 1.59304
[144]	valid_0's rmse: 1.59285
[145]	valid_0's rmse: 1.59275
[146]	valid_0's rmse: 1.59232
[147]	valid_

[54]	valid_0's rmse: 0.702398
[55]	valid_0's rmse: 0.699555
[56]	valid_0's rmse: 0.697297
[57]	valid_0's rmse: 0.69507
[58]	valid_0's rmse: 0.693887
[59]	valid_0's rmse: 0.691899
[60]	valid_0's rmse: 0.689783
[61]	valid_0's rmse: 0.687682
[62]	valid_0's rmse: 0.685709
[63]	valid_0's rmse: 0.683446
[64]	valid_0's rmse: 0.682098
[65]	valid_0's rmse: 0.680427
[66]	valid_0's rmse: 0.679003
[67]	valid_0's rmse: 0.677463
[68]	valid_0's rmse: 0.676503
[69]	valid_0's rmse: 0.675248
[70]	valid_0's rmse: 0.674776
[71]	valid_0's rmse: 0.673996
[72]	valid_0's rmse: 0.672946
[73]	valid_0's rmse: 0.672172
[74]	valid_0's rmse: 0.671309
[75]	valid_0's rmse: 0.670472
[76]	valid_0's rmse: 0.67046
[77]	valid_0's rmse: 0.6698
[78]	valid_0's rmse: 0.669614
[79]	valid_0's rmse: 0.668894
[80]	valid_0's rmse: 0.668527
[81]	valid_0's rmse: 0.668115
[82]	valid_0's rmse: 0.667872
[83]	valid_0's rmse: 0.667285
[84]	valid_0's rmse: 0.667507
[85]	valid_0's rmse: 0.666908
[86]	valid_0's rmse: 0.666662
[87]	valid_0's

[20]	valid_0's rmse: 1.56605
[21]	valid_0's rmse: 1.54893
[22]	valid_0's rmse: 1.53493
[23]	valid_0's rmse: 1.51731
[24]	valid_0's rmse: 1.50531
[25]	valid_0's rmse: 1.4904
[26]	valid_0's rmse: 1.47971
[27]	valid_0's rmse: 1.46899
[28]	valid_0's rmse: 1.46021
[29]	valid_0's rmse: 1.4527
[30]	valid_0's rmse: 1.44576
[31]	valid_0's rmse: 1.44216
[32]	valid_0's rmse: 1.43597
[33]	valid_0's rmse: 1.43176
[34]	valid_0's rmse: 1.4296
[35]	valid_0's rmse: 1.42613
[36]	valid_0's rmse: 1.42147
[37]	valid_0's rmse: 1.42252
[38]	valid_0's rmse: 1.41913
[39]	valid_0's rmse: 1.41383
[40]	valid_0's rmse: 1.41117
[41]	valid_0's rmse: 1.4112
[42]	valid_0's rmse: 1.4064
[43]	valid_0's rmse: 1.40443
[44]	valid_0's rmse: 1.40238
[45]	valid_0's rmse: 1.40215
[46]	valid_0's rmse: 1.40091
[47]	valid_0's rmse: 1.39944
[48]	valid_0's rmse: 1.40036
[49]	valid_0's rmse: 1.39796
[50]	valid_0's rmse: 1.39701
[51]	valid_0's rmse: 1.39513
[52]	valid_0's rmse: 1.39253
[53]	valid_0's rmse: 1.3903
[54]	valid_0's rmse:

In [103]:
for i in range(4):
    print('meter: '+ str(i))
    for model in meter_models[i]:
        print(model.best_score_['valid_0']['rmse'])


meter: 0
0.6766504792059432
0.48244439621719454
0.5961590142234228
0.6564252812330167
0.6608077473649704
meter: 1
1.220576659766382
1.1878621497244908
1.189461528316394
1.4753788509853354
1.3302926765867549
meter: 2
1.4664678217441978
1.3607795975980288
1.3463265302589502
1.5586578491718746
1.3788718583104924
meter: 3
1.4806505966429842
1.5387365830133888
1.2275170172686902
1.5580332777920087
1.6418494116108924


In [104]:
# Importance rank for first model in cross val models
for i in range(4):
    print('meter: '+ str(i))
    imprtc_df = pd.DataFrame()
    imprtc_df['feature'] = sample_train_X.drop('meter_reading_log1p', axis=1).columns   
    imprtc_df['importance'] = meter_models[i][0].feature_importances_
    imprtc_df.sort_values('importance', ascending=False, inplace= True)
    print(imprtc_df)


meter: 0
                          feature  importance
0                     building_id         660
3                  meter_0_median         276
2                    meter_0_mean         226
95         s_radiation_mean_lag72         133
100                          hour         108
99                      dayofweek         100
96          s_radiation_max_lag72          56
22                        site_id          50
64           s_radiation_max_lag3          40
98          s_radiation_std_lag72          38
5                     meter_0_max          31
6                     meter_0_std          25
69      air_temperature_min_lag72          24
37       air_temperature_min_lag3          20
67     air_temperature_mean_lag72          19
50     precip_depth_1_hr_std_lag3          19
88       wind_direction_max_lag72          18
65           s_radiation_min_lag3          17
90       wind_direction_std_lag72          16
23                    primary_use          15
66           s_radiation_

[101 rows x 2 columns]
meter: 3
                           feature  importance
0                      building_id        2508
95          s_radiation_mean_lag72         669
96           s_radiation_max_lag72         483
98           s_radiation_std_lag72         313
67      air_temperature_mean_lag72         284
17                    meter_3_mean         246
69       air_temperature_min_lag72         214
100                           hour         189
2                     meter_0_mean         162
37        air_temperature_min_lag3         138
36        air_temperature_max_lag3         127
68       air_temperature_max_lag72         117
35       air_temperature_mean_lag3         115
6                      meter_0_std         108
18                  meter_3_median          94
21                     meter_3_std          92
75      dew_temperature_mean_lag72          87
84    sea_level_pressure_max_lag72          86
50      precip_depth_1_hr_std_lag3          84
7                     meter_

In [44]:
# %%time
# ## Single fit single model

# gbm = LGBMRegressor(**gbm_params)
# f_train_X, f_train_y = getInFoldXY(train.index)
# gbm.fit(f_train_X, f_train_y)

In [45]:
# Generate test_X
test_X = x_pipes.transform(
    test
        .merge(building_pipes.transform(building), on='building_id', how='left').drop(['row_id'], axis=1)
        .merge(weather_pipes.transform(weather_test), on=['site_id', 'timestamp'], how='left')
    )

print(test_X.sample(n=20,  random_state=42))
print(test_X.shape)
print(test_X.dtypes)

         building_id meter  meter_0_mean  meter_0_median  meter_0_min  \
3573457          173     0      4.832031        4.785156     3.177734   
8315486          222     1      2.617188        2.638672     1.054688   
40305643        1354     2      0.000000        0.000000     0.000000   
16083617         712     0      1.302734        1.029297     0.000000   
37204119        1344     2      5.789062        5.761719     5.511719   
32144852        1119     1      4.425781        4.429688     0.000000   
5105044          249     0      5.921875        5.910156     4.453125   
36982844        1303     1      5.328125        6.355469     0.000000   
20487823         945     2      5.406250        5.531250     0.000000   
8404196          217     1      5.562500        5.515625     3.964844   
6889602          241     0      3.972656        3.908203     0.000000   
16963616         784     0      6.503906        6.460938     6.082031   
39666699        1381     2      4.281250        4.2

[20 rows x 92 columns]
(41697600, 92)
building_id                      category
meter                            category
meter_0_mean                      float16
meter_0_median                    float16
meter_0_min                       float16
meter_0_max                       float16
meter_0_std                       float16
meter_1_mean                      float16
meter_1_median                    float16
meter_1_min                       float16
meter_1_max                       float16
meter_1_std                       float16
meter_2_mean                      float16
meter_2_median                    float16
meter_2_min                       float16
meter_2_max                       float16
meter_2_std                       float16
meter_3_mean                      float16
meter_3_median                    float16
meter_3_min                       float16
meter_3_max                       float16
meter_3_std                       float16
site_id                          categ

In [46]:
def predMeters(test_X):
    test_y = test_X[['meter']] 
    test_y['meter_reading_log1p'] = np.nan
    for i in range(4):
        X = test_X[test_X['meter'] == i]
        if X.shape[0] > 0:
            preds = np.expm1(sum([model.predict(X) for model in meter_models[i]])/folds)
            test_y.loc[test_y['meter'] == i, 'meter_reading_log1p'] = preds
    return test_y['meter_reading_log1p'].tolist()
    
print(predMeters(test_X.sample(n=20,  random_state=42)))    


[138.31208164228642, 9.485521904545827, 49.973646503094734, 2.7080736972699198, 1105.1752086747995, 9.587553651427404, 332.90482886313845, 1234.5139043438674, 296.8153879184941, 96.20305506493278, 50.20849414041587, 666.4177910587814, 1055.705995072964, 37.15787025558416, 59.0464256955403, 62.4083513384531, 16.21357603947926, 73.57733366524941, 42.44589787652161, 166.6125611018554]


In [47]:
# Predict using cross val models ensemble 
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test_X.shape[0]/50000)))):
    res.append(predMeters(test_X.iloc[i:i+step_size]))
    i+=step_size
    gc.collect()


100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [38:13<00:00,  3.68s/it]


In [48]:
# Save using cross val models ensemble 
res = np.concatenate(res)
print(len(res))
submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_meter.csv.zip', index=False)
submission.shape

41697600


(41697600, 2)

In [49]:
# # Predict single model fit
# i=0
# res=[]
# step_size = 50000
# for j in tqdm(range(int(np.ceil(test_X.shape[0]/50000)))):
#    #res.append(np.expm1(sum([model.predict(test_X.iloc[i:i+step_size]) for model in models])/folds))
#    res.append(np.expm1(gbm.predict(test_X.iloc[i:i+step_size])))
#    i+=step_size
    